# Importing the model and the data

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

2025-10-28 17:13:41.702379: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761671621.986524      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761671622.069773      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load the trained model from Phase 2
base_model = tf.keras.models.load_model('/kaggle/input/ims-deeplearning-data-prep-part2/bearing_cnn_base_model.keras')
print("Base model loaded successfully!")
base_model.summary()

2025-10-28 17:14:00.355720: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Base model loaded successfully!


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 2034, 64)       │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 1017, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1017, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1008, 128)      │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 504, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 504, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 500, 128)       │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 250, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 250, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     2,048,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,639,749 (25.33 MB)

 Trainable params: 2,213,249 (8.44 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,426,500 (16.89 MB)

In [3]:
#Load the CWRU testing data
X_test = np.load('/kaggle/input/cwru-dataprep/X_CWRU_test_no_overlap.npy')
y_test = np.load('/kaggle/input/cwru-dataprep/y_CWRU_test_no_overlap.npy')

In [4]:
print(f"Class distribution: {np.bincount(y_test.astype(int))}")

Class distribution: [ 236 2133]


## Prepare CWRU for Transfer Learning

In [5]:
from sklearn.model_selection import train_test_split
#Split CWRU: 30% for fine-tuning, 70% for testing
X_cwru_train, X_cwru_test, y_cwru_train, y_cwru_test = train_test_split(
    X_test, y_test, 
    test_size=0.8, 
    stratify=y_test,
    random_state=42
)

print(f"CWRU Train: {X_cwru_train.shape}")
print(f"CWRU Test: {X_cwru_test.shape}")
print(f"Train distribution: {np.bincount(y_cwru_train.astype(int))}")
print(f"Test distribution: {np.bincount(y_cwru_test.astype(int))}")

CWRU Train: (473, 2048, 1)
CWRU Test: (1896, 2048, 1)
Train distribution: [ 47 426]
Test distribution: [ 189 1707]


## Transfer Learning (Freeze & Fine-tune)

In [6]:
print("=== EXPERIMENT 1: Transfer Learning ===")

# Clone the base model so we don't modify the original
model_transfer = keras.models.clone_model(base_model)
model_transfer.set_weights(base_model.get_weights())

# Freeze all layers except the last 3 (dense layers)
for layer in model_transfer.layers[:-3]:
    layer.trainable = False

print("Frozen layers:")
for i, layer in enumerate(model_transfer.layers):
    print(f"Layer {i}: {layer.name} - Trainable: {layer.trainable}")

# Recompile with same settings as original
model_transfer.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Recall()]
)

# Fine-tune on CWRU training data
print("\nFine-tuning on CWRU data...")
history_transfer = model_transfer.fit(
    X_cwru_train, y_cwru_train,
    epochs=10,
    batch_size=32,
    #validation_split=0.2,
    verbose=1
)



=== EXPERIMENT 1: Transfer Learning ===
Frozen layers:
Layer 0: conv1d - Trainable: False
Layer 1: max_pooling1d - Trainable: False
Layer 2: dropout - Trainable: False
Layer 3: conv1d_1 - Trainable: False
Layer 4: max_pooling1d_1 - Trainable: False
Layer 5: dropout_1 - Trainable: False
Layer 6: conv1d_2 - Trainable: False
Layer 7: max_pooling1d_2 - Trainable: False
Layer 8: dropout_2 - Trainable: False
Layer 9: flatten - Trainable: False
Layer 10: dense - Trainable: True
Layer 11: dropout_3 - Trainable: True
Layer 12: dense_1 - Trainable: True

Fine-tuning on CWRU data...
Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 96ms/step - accuracy: 0.5158 - loss: 215.2938 - recall: 0.4713
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.6139 - loss: 32.1051 - recall: 0.5975
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - accuracy: 0.8681 - loss: 1.6344 - recall: 0.9261
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 97ms/step - accuracy: 0.9211 - loss: 0.1369 - recall: 0.9910
Epoch 5/10

In [7]:
# Evaluate
y_pred_transfer = (model_transfer.predict(X_cwru_test) > 0.9).astype(int)

print("\nClassification Report:")
print(classification_report(y_cwru_test, y_pred_transfer))

cm_transfer = confusion_matrix(y_cwru_test, y_pred_transfer)
print("\nConfusion Matrix:")
print(cm_transfer)

print(f"\nPrediction breakdown:")
unique, counts = np.unique(y_pred_transfer, return_counts=True)
for cls, count in zip(unique, counts):
    percentage = (count / len(y_pred_transfer)) * 100
    print(f"Predicted class {cls}: {count} samples ({percentage:.1f}%)")

60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       189
           1       1.00      1.00      1.00      1707

    accuracy                           1.00      1896
   macro avg       1.00      0.99      0.99      1896
weighted avg       1.00      1.00      1.00      1896


Confusion Matrix:
[[ 185    4]
 [   0 1707]]

Prediction breakdown:
Predicted class 0: 185 samples (9.8%)
Predicted class 1: 1711 samples (90.2%)
